In [1]:
import os,sys,glob
import pandas as pd
import numpy as np
import nipype.interfaces.io as nio
import nipype.interfaces.fsl as fsl
import nipype.pipeline.engine as pe 
import nipype.algorithms.modelgen as model
import AEVHelpers as helper
import jtnipyutil.util as util #https://github.com/Jordan-Theriault/jtnipyutil
from nipype.interfaces.utility.wrappers import Function
from collections import namedtuple

In [2]:
subject_list=[101,102]
code_dir ='/Users/phoenix/Desktop/imaging_studies/AEV/Code/AEV_FirstLevel'
AEV_dir = '/Users/phoenix/Desktop/imaging_studies/AEV/Data'
fmri_data_path = 'AEV/BIDS_fmriprep/fmriprep/ses-01'
functional_folder_path = os.path.join(AEV_dir,fmri_data_path)


In [5]:
def CreateFirstLevelWorkFlow(subject_num,
                            regressor_directory,
                            functional_folder_path,
                            functional_scans_str):


    #get paths for scans
    functional_paths = os.path.join(functional_folder_path,functional_scans_str)
    functional_files = glob.glob(functional_paths)
    print(functional_files)
    mask_str=f'sub-{subject_num}/func/sub-{subject_num}_task-AEV_run-0*MNI*mask*.nii.gz'
    mask_paths = os.path.join(functional_folder_path,mask_str)
    mask_files = glob.glob(mask_paths)
    subject_info = helper.CreateSubjectInfoAllRuns(subject_num,
                                                   regressor_directory,
                                                   functional_folder_path, 
                                                   parametric=True)
    
    print('\n subject info \n\n',subject_info)
    contrast_non_parametric_regressors = ['heights', 'spiders','pain']
    contrast_parametric_regressors =  ['heights_parametric', 'spiders_parametric','pain_parametric']
    contrasts = [('heights', 'T', contrast_non_parametric_regressors, [1, 0,0]),
                 ('spiders', 'T', contrast_non_parametric_regressors, [0, 1,0]),
                 ('pain', 'T', contrast_non_parametric_regressors, [0, 0,1]),
                 ('heights_parametric', 'T',contrast_parametric_regressors, [1, 0,0]),
                 ('pain_parametric', 'T',contrast_parametric_regressors, [0, 0,1]),
                 ('spiders_parametric', 'T', contrast_parametric_regressors, [0, 1,0])]
    #define WF
    modelfit = pe.Workflow(name=f'firstlevel_parametric_{subject_num}',base_dir=code_dir+'/Test_FirstLevels')

    #define nodes
    model_spec = pe.Node(interface=model.SpecifyModel(
        input_units='secs',
        high_pass_filter_cutoff=128,
        time_repetition = 2.340,
        #functional_runs = functional_files, masking does this now!!!
        subject_info = subject_info
    ), name="modelspec")
    
    maskBold = pe.MapNode(Function(input_names=['img_file', 'mask_file'],
                                output_names=['out_files'],
                                function=util.mask_img),
                       iterfield = ['img_file', 'mask_file'],
                      name='maskBold')
    
    maskBold.inputs.img_file=functional_files
    maskBold.inputs.mask_file=mask_files
    
    level1design = pe.Node(interface=fsl.Level1Design(
        bases={'dgamma': {'derivs': False}},
        model_serial_correlations=True,
        interscan_interval =2.340,
        contrasts = contrasts
    ), name="level1design")

    feat_spec = pe.MapNode(
        interface=fsl.FEATModel(),
        name='modelgen',
        iterfield=['fsf_file', 'ev_files'])
    ## CHANGE THIS TO FEAT consistent with https://github.com/poldracklab/ds003-post-fMRIPrep-analysis/blob/master/workflows.py
    modelestimate = pe.MapNode(
        interface=fsl.FEAT(),
        mem_gb=10,
        name='modelestimate',
        iterfield=['fsf_file'])


    modelfit.connect([
        (maskBold, model_spec, [('out_files','functional_runs')]),
        (model_spec, level1design, [('session_info', 'session_info')]),
        (level1design, feat_spec, [('fsf_files', 'fsf_file'), ('ev_files',
                                                              'ev_files')]),
        (level1design, modelestimate, [('fsf_files', 'fsf_file')])
    ])
    
    return modelfit



In [7]:
for subject_num in subject_list:
    
    func_path = f'sub-{subject_num}/func/sub-{subject_num}_task-AEV_run-0*_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'
    
    confounds_path = os.path.join(AEV_dir,fmri_data_path,f'sub-{subject_num}/func/*confounds.tsv')
    
    
    single_subject = f"/Users/phoenix/Desktop/imaging_studies/AEV/Data/Behavioral_Data/AffVids_logfile_{subject_num}.txt"
    subject_behavioural = pd.read_csv(single_subject, sep=' ', skiprows=1,header= None)
    #helper.CreateRegressorTextFiles(subject_behavioural,code_dir,subject_num)
    
    first_level = CreateFirstLevelWorkFlow(subject_num,code_dir,functional_folder_path,func_path)
    first_level.run()

['/Users/phoenix/Desktop/imaging_studies/AEV/Data/AEV/BIDS_fmriprep/fmriprep/ses-01/sub-101/func/sub-101_task-AEV_run-01_bold_space-MNI152NLin2009cAsym_preproc.nii.gz', '/Users/phoenix/Desktop/imaging_studies/AEV/Data/AEV/BIDS_fmriprep/fmriprep/ses-01/sub-101/func/sub-101_task-AEV_run-02_bold_space-MNI152NLin2009cAsym_preproc.nii.gz', '/Users/phoenix/Desktop/imaging_studies/AEV/Data/AEV/BIDS_fmriprep/fmriprep/ses-01/sub-101/func/sub-101_task-AEV_run-03_bold_space-MNI152NLin2009cAsym_preproc.nii.gz']
ev array regressors
ev array regressors
ev array regressors
ev array regressors
ev array regressors
ev array regressors

 subject info 

 [Bunch(amplitudes=[[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0], [-0.362, 0.33299999999999996, -0.362, 0.391], [-0.044499999999999984, -0.12849999999999995, 0.05449999999999999, 0.11850000000000005], [-0.23025, 0.15675, 0.30375, -0.23025]], conditions=array(['heights', 'pain', 'spiders', 'heights_parametric',
       'pain_parametric', 

191202-08:27:20,410 nipype.workflow INFO:
	 Workflow firstlevel_parametric_102 settings: ['check', 'execution', 'logging', 'monitoring']
191202-08:27:20,422 nipype.workflow INFO:
	 Running serially.
191202-08:27:20,423 nipype.workflow INFO:
	 [Node] Setting-up "firstlevel_parametric_102.maskBold" in "/Users/phoenix/Desktop/imaging_studies/AEV/Code/AEV_FirstLevel/Test_FirstLevels/firstlevel_parametric_102/maskBold".
191202-08:27:20,431 nipype.workflow INFO:
	 [Node] "firstlevel_parametric_102.maskBold" found cached.
191202-08:27:20,433 nipype.workflow INFO:
	 [Node] Setting-up "firstlevel_parametric_102.modelspec" in "/Users/phoenix/Desktop/imaging_studies/AEV/Code/AEV_FirstLevel/Test_FirstLevels/firstlevel_parametric_102/modelspec".
191202-08:27:20,488 nipype.workflow INFO:
	 [Node] Cached "firstlevel_parametric_102.modelspec" - collecting precomputed outputs
191202-08:27:20,489 nipype.workflow INFO:
	 [Node] "firstlevel_parametric_102.modelspec" found cached.
191202-08:27:20,490 nipyp